In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['MALAYA_USE_HUGGINGFACE'] = 'true'

In [3]:
import malaya_speech
import json
import pickle
from tqdm import tqdm

In [4]:
with open('/home/husein/youtube/voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [5]:
with open('/home/husein/youtube/voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [6]:
model = malaya_speech.speaker_vector.deep_model(model = 'vggvox-v2')

In [7]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))  

In [8]:
unique_files[0]

'/home/husein/youtube/voxceleb-wav/aac-id04253-4kuv23gYxSE-00032.wav'

In [9]:
vectors = {}
for f in tqdm(unique_files):
    y, _ = malaya_speech.load(f)
    v = model.vectorize([y])[0]
    vectors[f] = v

 58%|█████▊    | 21089/36237 [1:12:29<53:42,  4.70it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 77%|███████▋  | 27922/36237 [1:35:47<41:44,  3.32it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 96%|█████████▌| 34760/36237 [1:59:39<04:59,  4.94it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`-

In [10]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    ls.append(labels[i][0])
    scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|██████████| 5900000/5900000 [00:51<00:00, 114721.61it/s]


In [11]:
len(scores)

5900000

In [12]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [13]:
calculate_eer(ls, scores)

(0.044600000000141, array(0.76090189))